# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_df = pd.read_csv('/Users/novakradovic/Git/API-challenge/output/city_data-2.csv')
vacation_df


,Unnamed: 0,City,Country,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Date
0,0,sioux lookout,ca,50.1001,-91.9170,24.41,41.0,0.0,2.34,1.627402e+09
1,1,kavieng,pg,-2.5744,150.7967,28.25,75.0,59.0,6.01,1.627402e+09
2,2,hilo,us,19.7297,-155.0900,22.84,95.0,76.0,0.56,1.627402e+09
3,3,vao,nc,-22.6667,167.4833,20.04,58.0,69.0,6.81,1.627402e+09
4,4,punta arenas,cl,-53.1500,-70.9167,6.06,84.0,90.0,4.12,1.627402e+09
...,...,...,...,...,...,...,...,...,...,...
563,565,brooks,ca,50.5834,-111.8851,26.10,30.0,92.0,5.23,1.627402e+09
564,566,sambava,mg,-14.2667,50.1667,21.69,76.0,21.0,5.44,1.627402e+09
565,567,am timan,td,11.0297,20.2827,24.72,80.0,69.0,4.29,1.627402e+09
566,568,zdvinsk,ru,54.7021,78.6610,13.69,64.0,66.0,1.87,1.627402e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
fig = gmaps.figure()
locations = vacation_df[['Latitude', 'Longitude']]
weights = vacation_df['Humidity']
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
new_df = vacation_df.loc[(vacation_df['Max Temperature']<27)&(vacation_df['Max Temperature']>22)
                         &(vacation_df['Cloudiness']==0)&(vacation_df['Wind Speed']<16)
                        &(vacation_df['Humidity']<60)].reset_index(drop=True)
hotel_df = new_df.drop(columns=['Unnamed: 0'])
hotel_df

,City,Country,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Date
0,sioux lookout,ca,50.1001,-91.9170,24.41,41.0,0.0,2.34,1.627402e+09
1,east london,za,-33.0153,27.9116,24.30,26.0,0.0,4.91,1.627402e+09
2,port alfred,za,-33.5906,26.8910,23.55,29.0,0.0,8.70,1.627402e+09
3,karaman,tr,37.0833,33.2500,26.53,30.0,0.0,10.09,1.627402e+09
4,timra,se,62.4870,17.3257,25.84,53.0,0.0,4.12,1.627402e+09
5,wilkie,ca,52.4168,-108.7014,25.97,33.0,0.0,3.77,1.627402e+09
6,opuwo,na,-18.0607,13.8400,24.19,17.0,0.0,3.49,1.627402e+09
7,balkhash,kz,46.8440,74.9804,23.72,34.0,0.0,3.68,1.627402e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
import pprint
hotel_df['Hotel Name'] = ''
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row['Latitude']
    long = row['Longitude']
    params["location"] = f"{lat},{long}"
    name_address = requests.get(base_url, params=params)
    name_address = name_address.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address['results'][0]["name"]
        
    except:
        print("Hotel within 5000m not found... skipping.")

# print(json.dumps(name_address, indent=4, sort_keys=True))
hotel_df

Hotel within 5000m not found... skipping.


,City,Country,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
0,sioux lookout,ca,50.1001,-91.9170,24.41,41.0,0.0,2.34,1.627402e+09,Moosehorn Lodge
1,east london,za,-33.0153,27.9116,24.30,26.0,0.0,4.91,1.627402e+09,Blue Lagoon Hotel
2,port alfred,za,-33.5906,26.8910,23.55,29.0,0.0,8.70,1.627402e+09,Oceana Beach Wildlife Reserve
3,karaman,tr,37.0833,33.2500,26.53,30.0,0.0,10.09,1.627402e+09,
4,timra,se,62.4870,17.3257,25.84,53.0,0.0,4.12,1.627402e+09,MästreBo
5,wilkie,ca,52.4168,-108.7014,25.97,33.0,0.0,3.77,1.627402e+09,Wilkie Motel
6,opuwo,na,-18.0607,13.8400,24.19,17.0,0.0,3.49,1.627402e+09,Opuwo Country Lodge
7,balkhash,kz,46.8440,74.9804,23.72,34.0,0.0,3.68,1.627402e+09,Гостиница Respect


In [6]:
#Just wanted to clean up the date
import datetime
from datetime import date
for i in hotel_df['Date']:
    timestamp = date.fromtimestamp(i)

hotel_df = hotel_df.drop(columns=['Date'])
hotel_df['Clean Date'] = timestamp
hotel_df

,City,Country,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name,Clean Date
0,sioux lookout,ca,50.1001,-91.9170,24.41,41.0,0.0,2.34,Moosehorn Lodge,2021-07-27
1,east london,za,-33.0153,27.9116,24.30,26.0,0.0,4.91,Blue Lagoon Hotel,2021-07-27
2,port alfred,za,-33.5906,26.8910,23.55,29.0,0.0,8.70,Oceana Beach Wildlife Reserve,2021-07-27
3,karaman,tr,37.0833,33.2500,26.53,30.0,0.0,10.09,,2021-07-27
4,timra,se,62.4870,17.3257,25.84,53.0,0.0,4.12,MästreBo,2021-07-27
5,wilkie,ca,52.4168,-108.7014,25.97,33.0,0.0,3.77,Wilkie Motel,2021-07-27
6,opuwo,na,-18.0607,13.8400,24.19,17.0,0.0,3.49,Opuwo Country Lodge,2021-07-27
7,balkhash,kz,46.8440,74.9804,23.72,34.0,0.0,3.68,Гостиница Respect,2021-07-27


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig_2 = gmaps.figure(layout=figure_layout)

markers = gmaps.marker_layer(locations)
fig_2.add_layer(markers)
fig_2

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [8]:
# Add marker layer ontop of heat map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))